In [1]:
'''
CC changed nothing of significance here. Just put in some print statements to discover that the problem 
was that there was only one file retrieved from the unzipping process on a couple of days and that 
prevented the code from running because it concatinates the files and uses the time between subsequent files to 
define the timestep

I managed to just unzip the files again and get the files to work
for all but one case. In that case the files were there but seemed to have an index error of some sort
so I put an error check in here just to be safe

        if len(c_files)==1:
            print("Skipping because regridding fails when only one time")
            continue 


'''

'\nCC changed nothing of significance here. Just put in some print statements to discover that the problem \nwas that there was only one file retrieved from the unzipping process on a couple of days and that \nprevented the code from running because it concatinates the files and uses the time between subsequent files to \ndefine the timestep\n\nI managed to just unzip the files again and get the files to work\nfor all but one case. In that case the files were there but seemed to have an index error of some sort\nso I put an error check in here just to be safe\n\n        if len(c_files)==1:\n            print("Skipping because regridding fails when only one time")\n            continue \n\n\n'

In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

# Standard Imports
%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import scipy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import os
import glob
import seaborn as sns
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

# ESIO Imports

from esio import EsioData as ed
from esio import import_data


In [2]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [3]:
E = ed.EsioData.load()
# Directories
all_models=['usnavygofs','usnavyncep','usnavysipn']
runType='forecast'
updateall = False

In [4]:
stero_grid_file = E.obs['NSIDC_0051']['grid']
obs_grid = import_data.load_grid_info(stero_grid_file, model='NSIDC')
# Ensure latitude is within bounds (-90 to 90)
# Have to do this because grid file has 90.000001
obs_grid['lat_b'] = obs_grid.lat_b.where(obs_grid.lat_b < 90, other = 90)

In [5]:
# Regridding Options
# method='conservative_normed' # ['bilinear', 'conservative', 'nearest_s2d', 'nearest_d2s', 'patch']
method = 'nearest_s2d'

In [6]:
# Set models that are different
var_dic = {'aice':'sic'}

In [7]:
#for model in ['usnavysipn']:
for model in all_models:
    print('Regridding ', model, '...')
    
    data_dir = E.model[model][runType]['native']
    data_out = E.model[model][runType]['sipn_nc']
    model_grid_file = E.model[model]['grid']
    
    # Files are stored as per time step (about 45 per init_time)
    # First parse files to see what unique init_times we have
    # ARCu0.08_121_2018042112_t0300.nc
    prefix = 'ARCu0'
    all_files = glob.glob(os.path.join(data_dir, '*'+prefix+'*.nc'))
    if model=='usnavygofs':
        init_N = 4
    else:
        init_N = 2
    init_times = list(set([s.split('_')[init_N] for s in all_files]))
    
    print("Found ",len(init_times)," initialization times.")
    if updateall:
        print("Updating all files...")
    else:
        print("Only updating new files")


    weights_flag = False # Flag to set up weights have been created

    # Load land/sea mask file
    if os.path.basename(model_grid_file)!='MISSING':
        ds_mask = xr.open_mfdataset(model_grid_file)
    else:
        ds_mask = None

    for cf in sorted(init_times):
        # Check if already imported and skip (unless updateall flag is True)
        f_out = os.path.join(data_out, prefix+'_'+cf+'_Stereo.nc') # netcdf file out 
        if not updateall:
            # TODO: Test if the file is openable (not corrupted)
            if os.path.isfile(f_out):
                print("Skipping ", cf, " already imported.")
                continue # Skip, file already imported

        c_files = sorted(glob.glob(os.path.join(data_dir, '*'+prefix+'*_'+cf+'*.nc')))

        if len(c_files)==1:
            print("Skipping because regridding fails when only one time")
            continue 

        # Some files have a "tau" variable that is hours since analysis
        try:
            ds = xr.open_mfdataset(c_files, concat_dim='time', decode_times=False, autoclose=True)
            # Format times
            ds.coords['init_time'] = np.datetime64(ds.tau.attrs['time_origin'])
            ds.coords['tau'] = ds.tau
            ds.swap_dims({'time':'tau'}, inplace=True)
            ds.rename({'tau':'fore_time'}, inplace=True)
            ds.fore_time.attrs['units'] = 'Forecast offset from initial time'
            ds = ds.drop(['time'])
            ds.coords['fore_time'] = ds.fore_time.astype('timedelta64[h]') 
            
        # Some do not
        except AttributeError:
            print('using alternative method for defining times')
            ds = xr.open_mfdataset(c_files, concat_dim='time', decode_times=True, autoclose=True)
            dt_mod = ds.time.values[1] - ds.time.values[0]
            ds.coords['init_time'] = ds.time.values[0] - dt_mod
            ds.coords['fore_time'] = ds.time - ds.init_time
            ds.swap_dims({'time':'fore_time'}, inplace=True);
            ds = ds.drop('time')
                        
        # Rename variables per esipn guidelines
        ds.rename(var_dic, inplace=True);
        
        # Apply masks (if available)
        if ds_mask:
            print('found mask')
            # land_mask is the fraction of native grid cell that is land
            # (1-land_mask) is fraction ocean
            # Multiply sic by fraction ocean to get actual native grid cell sic
            # Also mask land out where land_mask==1
            ds = ds * (1 - ds_mask.land_mask.where(ds_mask.land_mask<1))
            
        # Add mask variable so conservative regridding works as expected
        # DOESN"T WORK WITH OTHER METHODS!!
        #ds['mask'] = ds.sic.isel(fore_time=0).notnull() # Hardcoded variable choice
                        
        # Calculate regridding matrix
        regridder = xe.Regridder(ds, obs_grid, method, periodic=False, reuse_weights=weights_flag)

        weights_flag = True # Set true for following loops

        # Add NaNs to empty rows of matrix (forces any target cell with ANY source cells containing NaN to be NaN)
        #if method=='conservative':
        #    regridder = import_data.add_matrix_NaNs(regridder)

        # Regrid variables

        var_list = []
        for cvar in ds.data_vars:
            # 0 to NaN hack
            #offset = 10
            #da_coarse = regridder(ds[cvar]+10)
            #da_coarse = da_coarse.where(da_coarse>(offset)) - offset
            #var_list.append(da_coarse)
            
            # When doing nearest neighbor
            da_coarse = regridder(ds[cvar])
            var_list.append(da_coarse)
            
        ds_out = xr.merge(var_list)

        # Expand dims
        ds_out = import_data.expand_to_sipn_dims(ds_out)
                
        # # Save regridded to netcdf file
        ds_out.to_netcdf(f_out)
        
        ds_out = None # Memory clean up
        print('Saved ', f_out)

Regridding  usnavygofs ...
Found  588  initialization times.
Only updating new files
Skipping  2018010112  already imported.
Skipping  2018010212  already imported.
Skipping  2018010312  already imported.
Skipping  2018010412  already imported.
Skipping  2018010512  already imported.
Skipping  2018010612  already imported.
Skipping  2018010712  already imported.
Skipping  2018010812  already imported.
Skipping  2018010912  already imported.
Skipping  2018011012  already imported.
Skipping  2018011112  already imported.
Skipping  2018011212  already imported.
Skipping  2018011312  already imported.
Skipping  2018011412  already imported.
Skipping  2018011512  already imported.
Skipping  2018011612  already imported.
Skipping  2018011712  already imported.
Skipping  2018011812  already imported.
Skipping  2018011912  already imported.
Skipping  2018012012  already imported.
Skipping  2018012112  already imported.
Skipping  2018012212  already imported.
Skipping  2018012312  already impor

Skipping  2018100412  already imported.
Skipping  2018100512  already imported.
Skipping  2018100612  already imported.
Skipping  2018100712  already imported.
Skipping  2018100812  already imported.
Skipping  2018100912  already imported.
Skipping  2018101012  already imported.
Skipping  2018101112  already imported.
Skipping  2018101212  already imported.
Skipping  2018101312  already imported.
Skipping  2018101412  already imported.
Skipping  2018101512  already imported.
Skipping  2018101612  already imported.
Skipping  2018101712  already imported.
Skipping  2018101812  already imported.
Skipping  2018101912  already imported.
Skipping  2018102012  already imported.
Skipping  2018102112  already imported.
Skipping  2018102212  already imported.
Skipping  2018102312  already imported.
Skipping  2018102412  already imported.
Skipping  2018102512  already imported.
Skipping  2018102612  already imported.
Skipping  2018102712  already imported.
Skipping  2018102812  already imported.


Skipping  2019053012  already imported.
Skipping  2019053112  already imported.
Skipping  2019060112  already imported.
Skipping  2019060212  already imported.
Skipping  2019060312  already imported.
Skipping  2019060412  already imported.
Skipping  2019060512  already imported.
Skipping  2019060612  already imported.
Skipping  2019060712  already imported.
Skipping  2019060812  already imported.
Skipping  2019060912  already imported.
Skipping  2019061012  already imported.
Skipping  2019061112  already imported.
Skipping  2019061212  already imported.
Skipping  2019061312  already imported.
Skipping  2019061412  already imported.
Skipping  2019061512  already imported.
Skipping  2019061612  already imported.
Skipping  2019061712  already imported.
Skipping  2019061812  already imported.
Skipping  2019061912  already imported.
Skipping  2019062012  already imported.
Skipping  2019062112  already imported.
Skipping  2019062212  already imported.
Skipping  2019062312  already imported.


Skipping  2018111012  already imported.
Skipping  2018111112  already imported.
Skipping  2018111212  already imported.
Skipping  2018111312  already imported.
Skipping  2018111712  already imported.
Skipping  2018111812  already imported.
Skipping  2018111912  already imported.
Skipping  2018112012  already imported.
Skipping  2018112412  already imported.
Skipping  2018112512  already imported.
Skipping  2018112612  already imported.
Skipping  2018112712  already imported.
Skipping  2018120112  already imported.
Skipping  2018120212  already imported.
Skipping  2018120312  already imported.
Skipping  2018120412  already imported.
Skipping  2018120812  already imported.
Skipping  2018120912  already imported.
Skipping  2018121012  already imported.
Skipping  2018121112  already imported.
Skipping  2018121512  already imported.
Skipping  2018121612  already imported.
Skipping  2018121712  already imported.
Skipping  2018121812  already imported.
Skipping  2018122212  already imported.


using alternative method for defining times
Overwrite existing file: nearest_s2d_1251x4500_304x448.nc 
 You can set reuse_weights=True to save computing time.
Saved  /home/disk/sipn/nicway/data/model/usnavysipn/forecast/sipn_nc/ARCu0_2019070112_Stereo.nc
using alternative method for defining times
Reuse existing file: nearest_s2d_1251x4500_304x448.nc
Saved  /home/disk/sipn/nicway/data/model/usnavysipn/forecast/sipn_nc/ARCu0_2019070212_Stereo.nc
using alternative method for defining times
Reuse existing file: nearest_s2d_1251x4500_304x448.nc
Saved  /home/disk/sipn/nicway/data/model/usnavysipn/forecast/sipn_nc/ARCu0_2019070312_Stereo.nc
using alternative method for defining times
Reuse existing file: nearest_s2d_1251x4500_304x448.nc
Saved  /home/disk/sipn/nicway/data/model/usnavysipn/forecast/sipn_nc/ARCu0_2019070412_Stereo.nc
using alternative method for defining times
Reuse existing file: nearest_s2d_1251x4500_304x448.nc
Saved  /home/disk/sipn/nicway/data/model/usnavysipn/forecast/sipn

In [ ]:
# Clean up
if weights_flag:
    regridder.clean_weight_file()  # clean-up

# Plotting

In [ ]:
# sic_all = xr.open_mfdataset(f_out)

# # Set up plotting info
# cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
# cmap_sic.set_bad(color = 'red')

# # Plot original projection
# plt.figure(figsize=(20,10))
# ax1 = plt.axes(projection=ccrs.PlateCarree())
# ds_p = ds.sic.isel(fore_time=79)
# ds_p.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                  vmin=0, vmax=1,
#                                  cmap=matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10)),
#                     transform=ccrs.PlateCarree());
# ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
# gl = ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
# gl.xlabels_bottom = True
# gl.ylabels_left = True
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# ax1.coastlines(linewidth=0.75, color='black', resolution='50m');

# # Plot SIC on target projection
# (f, ax1) = ice_plot.polar_axis()
# ds_p2 = sic_all.sic.isel(init_time=0).isel(fore_time=79).isel(ensemble=0)
# ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                      transform=ccrs.PlateCarree(),
#                                      cmap=cmap_sic)
# ax1.set_title('Target Grid')
